In [ ]:
! pip install webdriver-manager
! pip install selenium


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from collections import Counter

# Function to scrape LinkedIn job postings and extract skills
def scrape_linkedin_jobs(keyword, num_pages):
    job_skills = []

    # Set up options for the Chrome WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Run Chrome in headless mode (optional)
    options.add_argument("--no-sandbox")

    # Start a Selenium WebDriver with options
    driver = webdriver.Chrome(options=options)


    # url = f'https://www.linkedin.com/jobs/search/?keywords={keyword}&location=Bengaluru%2C%20Karnataka%2C%20India'
    url = f'https://www.linkedin.com/jobs/search/distance=25&f_WT=2&geoId=102713980&keywords=golang&origin=JOB_SEARCH_PAGE_JOB_FILTER'
    driver.get(url)
    j = 0
    # Scroll to load more jobs (you may need to adjust the number of scrolls)
    for _ in range(num_pages):
        print("scroll ######",j)
        j = j+1
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        time.sleep(2)  # Wait for content to load

    # Extract job titles and skills (modify as needed)
    job_cards = driver.find_elements(By.CSS_SELECTOR, '.base-card')
    break_element = 0
    for card in job_cards:
        try:

            job_title_element = WebDriverWait(card, 10)\
            .until(EC.presence_of_element_located((By.CSS_SELECTOR, '.base-search-card__title')))
            company_element = WebDriverWait(card, 10)\
            .until(EC.presence_of_element_located((By.CSS_SELECTOR, '.base-search-card__subtitle')))
            description = WebDriverWait(card, 10)\
            .until(EC.presence_of_element_located((By.CSS_SELECTOR, '.base-card__full-link')))


            company_name = company_element.text
            job_title = job_title_element.text
            print(job_title, " ", company_name)
            job_link = description.get_attribute('href')
            print(job_link)

            #Hitting each job's URL to get more information
            job_driver = webdriver.Chrome(options=options)
            job_driver.get(job_link)

            #expand descriptions by clicking on show more
            expand_description(job_driver)
            #extract description element
            job_description = extract_job_description(job_driver)
            #extract skills from description
            extracted_skills = extract_skills(job_description)
            job_skills.append(extracted_skills)
            job_driver.quit()

        except Exception as e:
            print("Job details not found for this card.")
            continue

    # Close the WebDriver when done
    driver.quit()

    #print(job_skills)

    return job_skills


# Function to extract job description using JavaScript
def extract_job_description(driver):
    try:
        # Wait for the job description element to be present (you can adjust the timeout)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.description')))

        # Execute JavaScript code to extract job description
        job_description = driver.execute_script("return document.querySelector('.description').textContent")
        return job_description
    except TimeoutException:
        return "Job description not found or couldn't be loaded"

# Function to expand job description by clicking "show more" if available
def expand_description(driver):
    try:
        show_more_button = WebDriverWait(card, 10)\
            .until(EC.presence_of_element_located((By.CSS_SELECTOR, '.show-more-less-html__button')))

        show_more_button.click()
        time.sleep(2)  # Wait for the description to expand
    except Exception as e:
        pass  # No "show more" button found or error occurred


# Function to extract skills from a job title
def extract_skills(description):
    # This is a basic example, you can extend this to match more skills
    # skills = [
    # "Sourcing",
    # "Recruitment",
    # "Talent Acquisition",
    # "Job Portals",
    # "ATS (Applicant Tracking System)",
    # "Screening",
    # "Interviewing",
    # "Candidate Assessment",
    # "Onboarding",
    # "Employee Referral Programs",
    # "Job Descriptions",
    # "Negotiation",
    # "Compliance",
    # "Diversity and Inclusion",
    # "HR Metrics",
    # "Candidate Experience",
    # "Networking",
    # "Market Research",
    # "Communication",
    # "Time Management",
    # "Multitasking",
    # "Data Analysis",
    # "HR Software",
    # "Behavioral Interviewing",
    # "Employer Branding",
    # "Feedback",
    # "Reference Checking",
    # "Stakeholder Management",
    # "Adaptability",
    # "Conflict Resolution"]
    skills = ["golang"]
    description = description.lower()
    skills_list = [skill for skill in skills if skill.lower() in description]
    print(skills_list)
    return skills_list

# Main function
if __name__ == "__main__":
    # keyword = "us%20it%20recuiter"
    keyword = "golang%20developer"
    num_pages = 20 # You can adjust the number of pages to scrape

    job_skills = scrape_linkedin_jobs(keyword, num_pages)

    print(f'{keyword} jobs: {len(job_skills)}')

    flattened_skills = [skill for sublist in job_skills for skill in sublist]
    skill_counts = Counter(flattened_skills)
    top_skills = skill_counts.most_common(30)

    for skill, count in top_skills:
        print(f'{skill}: {count}')

scroll ###### 0
scroll ###### 1
scroll ###### 2
scroll ###### 3
scroll ###### 4
scroll ###### 5
scroll ###### 6
scroll ###### 7
scroll ###### 8
scroll ###### 9
scroll ###### 10
scroll ###### 11
scroll ###### 12
scroll ###### 13
scroll ###### 14
scroll ###### 15
scroll ###### 16
scroll ###### 17
scroll ###### 18
scroll ###### 19
Software Engineer (Junior Backend)   Clarifai
https://in.linkedin.com/jobs/view/software-engineer-junior-backend-at-clarifai-3871750365?position=1&pageNum=0&refId=vOKx%2FyWNmBlzZ1A%2BpyRgfw%3D%3D&trackingId=HM9V5mnUP%2BTkwJNqDo%2FSTA%3D%3D&trk=public_jobs_jserp-result_search-card
[]
Backend Developer   CRED
https://in.linkedin.com/jobs/view/backend-developer-at-cred-3861556491?position=2&pageNum=0&refId=vOKx%2FyWNmBlzZ1A%2BpyRgfw%3D%3D&trackingId=8vo8er25RVKRpeiuH6ELoQ%3D%3D&trk=public_jobs_jserp-result_search-card
[]
Back End Developer (Java/ Golang)   Arista Networks
https://in.linkedin.com/jobs/view/back-end-developer-java-golang-at-arista-networks-3845617658

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class RecordRetrievalRequest:
    def __init__(self, client_name, record_type):
        self.client_name = client_name
        self.record_type = record_type
        self.records = []

# 1. Initiate and manage the record retrieval process by contacting relevant parties
def initiate_record_retrieval(client_name, record_type):
    # Contact relevant parties to request records
    # ...

    # Obtain necessary authorization forms from clients or other authorized individuals
    # ...

    # Coordinate with external parties to ensure timely retrieval of records
    # ...

    # Return a record retrieval request object
    return RecordRetrievalRequest(client_name, record_type)

# 2. Request and obtain necessary authorization forms from clients or other authorized individuals
def request_authorization_forms(client_name, record_type):
    # Send authorization form request to the client or authorized individual
    # ...

    # Track the status of the authorization form request
    # ...

    # Return the authorization form when it is received
    return authorization_form

# 3. Coordinate with external parties to ensure timely retrieval of records within established deadlines
def coordinate_record_retrieval(record_retrieval_request):
    # Communicate with external parties to schedule record retrieval
    # ...

    # Track the status of the record retrieval process
    # ...

    # Notify the client when records are retrieved
    # ...
    return ""

# 4. Review retrieved records for accuracy
def review_records(records):
    # Review records for completeness and accuracy
    # ...

    # Flag any discrepancies or errors in the records
    # ...

    # Return the reviewed records
    return reviewed_records

# Main function
def main():
    # Initiate the record retrieval process
    record_retrieval_request = initiate_record_retrieval("John Doe", "Medical Records")

    # Request and obtain necessary authorization forms
    authorization_form = request_authorization_forms("John Doe", "Medical Records")

    # Coordinate with external parties to ensure timely retrieval of records
    coordinate_record_retrieval(record_retrieval_request)

    # Review retrieved records for accuracy
    reviewed_records = review_records(record_retrieval_request.records)

    # Print the reviewed records
    print(reviewed_records)

if __name__ == "__main__":
    main()


NameError: name 'authorization_form' is not defined